In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl
import time
import os

AttributeError: partially initialized module 'polars' has no attribute '_cpu_check' (most likely due to a circular import)

In [2]:
!lscpu

Architecture:                         x86_64
CPU op-mode(s):                       32-bit, 64-bit
Byte Order:                           Little Endian
Address sizes:                        46 bits physical, 48 bits virtual
CPU(s):                               4
On-line CPU(s) list:                  0-3
Thread(s) per core:                   2
Core(s) per socket:                   2
Socket(s):                            1
NUMA node(s):                         1
Vendor ID:                            GenuineIntel
CPU family:                           6
Model:                                79
Model name:                           Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:                             0
CPU MHz:                              2200.140
BogoMIPS:                             4400.28
Hypervisor vendor:                    KVM
Virtualization type:                  full
L1d cache:                            64 KiB
L1i cache:                            64 KiB
L2 cache:                  

In [7]:
!nvidia-smi


Sun Oct  6 10:46:43 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
start_time = time.time()
products_df = pl.read_csv("/kaggle/input/instacart-market-basket-analysis/products.csv")
departments_df = pl.read_csv("/kaggle/input/instacart-market-basket-analysis/departments.csv")
aisles_df = pl.read_csv("/kaggle/input/instacart-market-basket-analysis/aisles.csv")
orders_df = pl.read_csv("/kaggle/input/instacart-market-basket-analysis/orders.csv")
train_orders = pl.read_csv("/kaggle/input/instacart-market-basket-analysis/order_products__train.csv")
prior_orders = pl.read_csv("/kaggle/input/instacart-market-basket-analysis/order_products__prior.csv")
time.time() - start_time

4.619037866592407

In [3]:
departments_df.select(['department']).to_series().to_list()

['frozen',
 'other',
 'bakery',
 'produce',
 'alcohol',
 'international',
 'beverages',
 'pets',
 'dry goods pasta',
 'bulk',
 'personal care',
 'meat seafood',
 'pantry',
 'breakfast',
 'canned goods',
 'dairy eggs',
 'household',
 'babies',
 'snacks',
 'deli',
 'missing']

In [4]:
products_df.collect_schema()

Schema([('product_id', Int64),
        ('product_name', String),
        ('aisle_id', Int64),
        ('department_id', Int64)])

In [5]:
products_df.group_by('department_id').agg([
    pl.col('product_name')
]).sort('department_id')

department_id,product_name
i64,list[str]
1,"[""Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce"", ""Cut Russet Potatoes Steam N' Mash"", … ""En Croute Roast Hazelnut Cranberry""]"
2,"[""Camilia, Single Liquid Doses"", ""Arrowroot Powder"", … ""Lavender & Bergamot Sleep Balm""]"
3,"[""Whole Wheat Tortillas"", ""100% Whole Wheat Pita Bread"", … ""Artisan Baguette""]"
4,"[""White Pearl Onions"", ""Organic Clementines"", … ""Cucumber Kirby""]"
5,"[""Mirabelle Brut Rose"", ""Chardonnay Paso Robles"", … ""Vodka, Triple Distilled, Twist of Vanilla""]"
…,…
17,"[""Fresh Scent Dishwasher Cleaner"", ""School Glue, Washable, No Run"", … ""Apple Cinnamon Scented Candles""]"
18,"[""Overnight Diapers Size 6"", ""Banana & Sweet Potato Organic Teething Wafers"", … ""Sweet Potatoes Stage 2""]"
19,"[""Chocolate Sandwich Cookies"", ""Mint Chocolate Flavored Syrup"", … ""Famous Chocolate Wafers""]"


In [6]:
products_df.select([pl.col(col).n_unique().alias(col+'_n_uniques') for col in products_df.columns])
products_df.group_by('department_id').agg([
    pl.col(col).n_unique().alias(col+'_n_uniques')  for col in products_df.columns 
]).drop('department_id_n_uniques').sort('department_id')


department_id,product_id_n_uniques,product_name_n_uniques,aisle_id_n_uniques
i64,u32,u32,u32
1,4007,4007,11
2,548,548,1
3,1516,1516,5
4,1684,1684,5
5,1054,1054,5
…,…,…,…
17,3085,3085,10
18,1081,1081,4
19,6264,6264,11


In [7]:
orders_df.collect_schema()

Schema([('order_id', Int64),
        ('user_id', Int64),
        ('eval_set', String),
        ('order_number', Int64),
        ('order_dow', Int64),
        ('order_hour_of_day', Int64),
        ('days_since_prior_order', Float64)])

In [8]:
train_orders.collect_schema()

Schema([('order_id', Int64),
        ('product_id', Int64),
        ('add_to_cart_order', Int64),
        ('reordered', Int64)])

In [9]:
prior_orders.collect_schema()

Schema([('order_id', Int64),
        ('product_id', Int64),
        ('add_to_cart_order', Int64),
        ('reordered', Int64)])

In [10]:
merged_prior_orders = (
    prior_orders
    .join(
        products_df,
        on="product_id",
        how="left"
    )
    .join(
        orders_df,
        on="order_id",
        how="left"
    )
    .drop("eval_set")  # Drop the column if it's present
)
merged_prior_orders.collect_schema()

Schema([('order_id', Int64),
        ('product_id', Int64),
        ('add_to_cart_order', Int64),
        ('reordered', Int64),
        ('product_name', String),
        ('aisle_id', Int64),
        ('department_id', Int64),
        ('user_id', Int64),
        ('order_number', Int64),
        ('order_dow', Int64),
        ('order_hour_of_day', Int64),
        ('days_since_prior_order', Float64)])

In [11]:
merged_train_orders = (
    train_orders.join(
        orders_df,
        on='order_id',
        how='left'
    )
    .join(
        products_df,
        on='product_id',
        how='left'
    ).drop('eval_set')
)

In [12]:
# How often the user reordered items
merged_prior_orders.group_by(['user_id']).agg([
    (pl.col('reordered') == 1).sum()
]).sort('user_id')

user_id,reordered
i64,u32
1,41
2,93
3,55
4,1
5,14
…,…
206205,8
206206,135
206207,131


In [13]:
# Time of day the user visits

merged_prior_orders.group_by(["user_id", "order_hour_of_day"]).agg([
    pl.col("order_id").n_unique().alias("visit_frequency")  # Count unique orders per user per hour
]).sort("visit_frequency",descending=True).group_by("user_id").agg([
    pl.col("visit_frequency").max().alias("max_visit_frequency"),  # Get max visit frequency
    pl.col("order_hour_of_day").first().alias("max_visit_hour")  # Get the first hour with the max frequency
]).sort("user_id")



user_id,max_visit_frequency,max_visit_hour
i64,u32,i64
1,3,7
2,5,10
3,4,16
4,2,11
5,2,18
…,…,…
206205,1,12
206206,15,18
206207,3,12


In [14]:
# Whether the user ordered organic, gluten-free, or Asian items in the past
merged_prior_orders.group_by(['user_id','order_id']).agg([
    pl.col("product_name").alias("list_of_products")
]).with_columns([
     pl.col('list_of_products').list.eval(pl.element().str.contains("(?i)Organic|(?i)Gluten-Free|(?i)Asian")).list.any().alias("orderd_asian_organic_gluten_free")
]).sort("user_id",'order_id')

user_id,order_id,list_of_products,orderd_asian_organic_gluten_free
i64,i64,list[str],bool
1,431534,"[""Soda"", ""Original Beef Jerky"", … ""Bag of Organic Bananas""]",true
1,473747,"[""Soda"", ""Original Beef Jerky"", … ""Creamy Almond Butter""]",true
1,550135,"[""Soda"", ""Pistachios"", … ""Cinnamon Toast Crunch""]",true
1,2254736,"[""Soda"", ""Original Beef Jerky"", … ""XL Pick-A-Size Paper Towel Rolls""]",true
1,2295261,"[""Organic Half & Half"", ""Zero Calorie Cola"", … ""Original Beef Jerky""]",true
…,…,…,…
206209,2307371,"[""Calcium Enriched 100% Lactose Free Fat Free Milk"", ""Diet Pepsi Pack"", ""Bag of Organic Bananas""]",true
206209,2558525,"[""Ultra Downy® Clean Breeze™ Liquid Fabric Conditioner 51 Fl oz. 60 loads Fabric Enhancers"", ""Original High Efficiency Liquid Laundry Detergent"", ""Diet Pepsi Pack""]",false
206209,2977660,"[""Banana"", ""Calcium Enriched 100% Lactose Free Fat Free Milk"", … ""Roasted & Salted Shelled Pistachios""]",true


In [15]:
# Features based on order sizes
merged_prior_orders.group_by([
    'user_id','order_id'
]).agg([
    pl.col("product_id").alias("list_of_products")
]).with_columns([
    pl.col('list_of_products').list.eval(pl.element().len()).list.first().alias("count_of_products")
]).group_by("user_id").agg([
    pl.col("count_of_products").mean().alias("average_basket_size"),
    pl.col("count_of_products").min().alias("min_basket_size"),
    pl.col("count_of_products").max().alias("max_basket_size"),
    pl.col("count_of_products").median().alias("median_basket_size")
]).sort(["user_id"])


user_id,average_basket_size,min_basket_size,max_basket_size,median_basket_size
i64,f64,u32,u32,f64
1,5.9,4,9,5.5
2,13.928571,5,26,13.5
3,7.333333,5,11,7.0
4,3.6,2,7,3.0
5,9.25,5,12,10.0
…,…,…,…,…
206205,10.666667,7,17,8.0
206206,4.253731,1,24,4.0
206207,13.9375,5,25,13.0


In [16]:
# How many of the user’s orders contained no previously purchased items
merged_prior_orders.group_by([
    "user_id","order_id"
]).agg([
    pl.col('reordered').cast(pl.Boolean).alias("reordered_list")
]).with_columns([
    pl.col("reordered_list").list.eval(pl.element().any()).list.first().alias("contains_reordered_item")
]).group_by("user_id").agg([
    pl.col("order_id").len().alias("total_orderes"),
    pl.col('contains_reordered_item').sum()
]).with_columns([
        (pl.col("total_orderes")-pl.col("contains_reordered_item")).alias("not_contains_reordered_item")
]).sort("user_id")

user_id,total_orderes,contains_reordered_item,not_contains_reordered_item
i64,u32,u32,u32
1,10,9,1
2,14,12,2
3,12,11,1
4,5,1,4
5,4,3,1
…,…,…,…
206205,3,2,1
206206,67,63,4
206207,16,15,1


In [17]:
#  How often the item is purchased

merged_prior_orders.group_by("product_id").agg([
    pl.col("order_id").len()
]).sort('product_id')

product_id,order_id
i64,u32
1,1852
2,90
3,277
4,329
5,15
…,…
49684,9
49685,49
49686,120


In [18]:
 # Position in the cart

merged_prior_orders.group_by("product_id","add_to_cart_order").agg([
    pl.col("order_id").len().alias("total_frequency"),
]).sort("total_frequency",descending=True).group_by("product_id").agg([
    pl.col("add_to_cart_order").first().alias("frequent_position"),
    pl.col("total_frequency").max().alias("frequeny")
]).sort("product_id")

product_id,frequent_position,frequeny
i64,i64,u32
1,1,272
2,1,11
3,1,43
4,7,29
5,7,4
…,…,…
49684,2,3
49685,1,8
49686,3,16


In [19]:
# How many users buy it as "one shot" item
merged_prior_orders.group_by("order_id").agg([
    pl.col("product_id").alias("list_of_products")
]).with_columns([
    (pl.col("list_of_products").list.len()==1).alias("is_one_shot")
]).explode("list_of_products").group_by("list_of_products").agg([
    pl.col("is_one_shot").sum().alias("frequency_of_one_shot_purchased")
]).sort("list_of_products")

list_of_products,frequency_of_one_shot_purchased
i64,u32
1,43
2,0
3,0
4,0
5,0
…,…
49684,0
49685,0
49686,0


In [20]:
# Stats on the number of items that co-occur with this item
merged_prior_orders.select(['order_id','product_id']).join(
    merged_prior_orders.select(['order_id','product_id']).rename({'product_id':'product_id2'}),
    how='inner',
    on='order_id'
).filter(pl.col("product_id") < pl.col("product_id2")).group_by([
    "product_id","product_id2"
]).agg([
    pl.len().alias("frequency")
]).sort("frequency",descending=True).group_by("product_id").agg([
    pl.col("product_id2").first().alias("most_frequent_item"),
    pl.col("frequency").first().alias("most_frequent_count")
]).sort("product_id")


product_id,most_frequent_item,most_frequent_count
i64,i64,u32
1,6184,258
2,13176,13
3,13176,48
4,4846,79
5,2846,5
…,…,…
49678,49683,13
49679,49683,2
49680,49683,74


In [21]:
start_time=time.time()
# Define a function to calculate streak lengths
def calculate_streak_lengths(is_streak):
    streak_lengths = []
    length = 0
    for streak in is_streak:
        if streak:
            length += 1
        else:
            if length > 0:
                streak_lengths.append(length)
                length = 0
    if length > 0:
        streak_lengths.append(length)
    return streak_lengths

df = merged_prior_orders.sort("user_id","product_id","order_number").with_columns([
    ((pl.col('user_id') == pl.col('user_id').shift(1)) & 
     (pl.col('product_id') == pl.col('product_id').shift(1)) & 
     ((pl.col('order_number') - pl.col('order_number').shift(1)) == 1)
    ).alias('is_streak')
])


# Apply streak length calculation to each group
streaks = (
    df.group_by(['user_id', 'product_id'])
    .agg([
        pl.col('is_streak').map_elements(lambda x: calculate_streak_lengths(x), return_dtype=pl.List(pl.Int64)).alias('streak_lengths')
    ])
)

# Flatten and analyze streak lengths
flattened_streaks = (
    streaks.explode('streak_lengths')
    .group_by(['user_id', 'product_id'])
    .agg([
        pl.col('streak_lengths').mean().alias('mean_streak_length'),
        pl.col('streak_lengths').min().alias('min_streak_length'),
        pl.col('streak_lengths').max().alias('max_streak_length'),
        pl.col('streak_lengths').median().alias('median_streak_length'),
        pl.col('streak_lengths').count().alias('streak_count')
    ])
)

flattened_streaks = flattened_streaks.fill_null(0)
end_time=time.time()-start_time
print(end_time)

166.40949320793152


In [22]:
flattened_streaks.sort("user_id","product_id")

user_id,product_id,mean_streak_length,min_streak_length,max_streak_length,median_streak_length,streak_count
i64,i64,f64,i64,i64,f64,u32
1,196,9.0,9,9,9.0,1
1,10258,8.0,8,8,8.0,1
1,10326,0.0,0,0,0.0,0
1,12427,9.0,9,9,9.0,1
1,13032,0.0,0,0,0.0,0
…,…,…,…,…,…,…
206209,43961,0.0,0,0,0.0,0
206209,44325,0.0,0,0,0.0,0
206209,48370,0.0,0,0,0.0,0


In [23]:

# Set seed for reproducibility
np.random.seed(0)

# Define number of users, orders, and products per order
num_users = 10
orders_per_user = 5
products_per_order = 3  # Number of products per order

# Create lists for the dataset
user_ids = []
order_ids = []
order_numbers = []
product_ids = []

# Generate data
for user_id in range(1, num_users + 1):
    for order_number in range(1, orders_per_user + 1):
        order_id = np.random.randint(1000, 2000)  # Generate random order_id once per order
        for _ in range(products_per_order):
            user_ids.append(user_id)
            order_ids.append(order_id)  # Same order_id for the same order
            order_numbers.append(order_number)
            product_ids.append(user_id * 10 + np.random.randint(1, 100))  # Random product_id for each product

# Create a DataFrame
df = pl.DataFrame({
    'user_id': user_ids,
    'order_id': order_ids,
    'order_number': order_numbers,
    'product_id': product_ids
})

# Show the DataFrame
print(df)


shape: (150, 4)
┌─────────┬──────────┬──────────────┬────────────┐
│ user_id ┆ order_id ┆ order_number ┆ product_id │
│ ---     ┆ ---      ┆ ---          ┆ ---        │
│ i64     ┆ i64      ┆ i64          ┆ i64        │
╞═════════╪══════════╪══════════════╪════════════╡
│ 1       ┆ 1684     ┆ 1            ┆ 58         │
│ 1       ┆ 1684     ┆ 1            ┆ 75         │
│ 1       ┆ 1684     ┆ 1            ┆ 78         │
│ 1       ┆ 1763     ┆ 2            ┆ 78         │
│ 1       ┆ 1763     ┆ 2            ┆ 20         │
│ …       ┆ …        ┆ …            ┆ …          │
│ 10      ┆ 1889     ┆ 4            ┆ 157        │
│ 10      ┆ 1889     ┆ 4            ┆ 117        │
│ 10      ┆ 1152     ┆ 5            ┆ 130        │
│ 10      ┆ 1152     ┆ 5            ┆ 122        │
│ 10      ┆ 1152     ┆ 5            ┆ 126        │
└─────────┴──────────┴──────────────┴────────────┘


In [24]:
result = (
    df
    .select("user_id", "product_id", "order_number")
    .sort("user_id", "order_number")
    .group_by("user_id", "order_number")
    .agg([
        pl.col("product_id").list().alias("list_of_products")
    ])
)

TypeError: 'ExprListNameSpace' object is not callable

In [ ]:
df.sort("user_id","order_number").group_by_dynamic(
    group_by=["user_id"],
    index_column=pl.col('order_number').alias("index"),
    period="3i",
    every="3i",
    closed='left',
    label='left',
    start_by='datapoint'
).agg([
    pl.col("product_id").alias('list_of_products'),
    pl.col("product_id").len().alias("length")
]).sort("user_id","index")
